In [ ]:
import re
import csv
import docx
import numpy as np

from docx import Document

In [ ]:
c = 8
yshift = 4
xshift = 2

# Koordinater
row_avd        = 2
row_avd_rolle  = 3
row_rolle      = 4
row_data_start = 6

row_hensikt    = 8 - yshift
row_overord    = 9 - yshift
row_underord   = 10 - yshift
row_standard   = 15 - yshift

col_underaktivitet = 3 
col_data_start     = 4
col_data_end       = 4 + c
#col_data_end       = 60

col_beskr_start    = 3 - xshift
col_beskr_end      = 3 + c - xshift
col_standard       = 2 - xshift

In [ ]:
rowset = []
with open('HUKI for prosesser i Feiring Bruk v3.csv', 'rb') as csvfile:
    #dialect = csv.Sniffer().sniff(csvfile.read(1024))
    csvfile.seek(0)
    reader = csv.reader(csvfile, delimiter=';')
    
    rowset = []
    i = 0
    for row in reader:
        if i < 20: rowset.append(row) # Remove filter
        i += 1
        
D = np.array(rowset,dtype=np.string_)

with open('HUKI for prosesser i Feiring Bruk v3 beskrivelser.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    rowset = []
    for row in reader:
        rowset.append(row)

D1 = np.array(rowset,dtype=np.string_)
    

rolle     = D[row_rolle, col_data_start:col_data_end+1]
avd_rolle = D[row_avd_rolle, col_data_start:col_data_end+1]
desc      = D[row_data_start:, col_underaktivitet]
data      = D[row_data_start:, col_data_start:col_data_end+1]

hensikt   = D1[row_hensikt, col_beskr_start:col_beskr_end+1]
overord   = D1[row_overord, col_beskr_start:col_beskr_end+1]
underord  = D1[row_underord, col_beskr_start:col_beskr_end+1]
standard  = D1[row_standard, col_standard]

(m,n) = data.shape

ansvar = ['H','U','K','I']
ansvartext = {'H': 'Rollen er hovedansvarlig for at følgende aktiviteter blir utført, og for å ta beslutninger knyttet til disse aktivitetene',
              'U': 'Rollen skal utføre følgende aktiviteter',
              'K': 'Rollen skal være tilgjengelig for konsultasjon i følgende aktiviteter',
              'I': 'Rollen skal informeres om følgende aktiviteter'} 
    

In [ ]:
def stringprep(s):
    return unicode(s.decode('utf-8')) # document must be saved in utf-8..

def stringclean(s):
    tmp = re.search('([a-zæøåA-zÆØÅ0-9 _]*)',s)
    return unicode(tmp.group(0).decode('utf-8'))
    

def print_document(doc):
    if doc['ansvar']:
        print '\n--------', doc['rolle'], '--------'
        for a in [aa for aa in ansvar if aa in doc['ansvar']]:
            print '\n--', a, '--'
            for d in doc['ansvar'][a].values():
                print ' * ', d['desc']
                if a=='U':
                    if d['konsultere']: print '    Konsultere:', ','.join(d['konsultere'])
                    if d['informere']:  print '    Informere: ', ','.join(d['informere'])
                    
    pass

def generate_document(doc):
    
    document = Document('blank.docx')
    
    print stringprep(doc['rolle'])
    
    #Overskrift med rollenavn
    document.add_paragraph(stringprep(doc['rolle']), style='Heading 1')
    document.add_paragraph('\n\n')

    # Tekst om rollens hensikt
    document.add_paragraph(u'Rollens hensikt', style='Heading 3')
    document.add_paragraph(stringprep(doc['hensikt']))
    
    # Tabell med rapportering
    document.add_paragraph(u'Rapportering', style='Heading 3')
    
    table = document.add_table(rows=1, cols=2, style='Table Grid BE-style')
    hdr = table.rows[0].cells
    hdr[0].text = stringprep('Nærmeste overordnede til rollen:')
    hdr[1].text = stringprep('Rollen er nærmeste overordnede til følgende roller:')
    row = table.add_row().cells
    row[0].text = stringprep(doc['overordnet'])
    row[1].text = stringprep(doc['underordnet'])
    
    # HUKI
    document.add_paragraph(u'Ansvar og myndighet', style='Heading 3')
    
    for a in ansvar:
        
        document.add_paragraph(stringprep(ansvartext[a]), style='Heading 4')
        
        # List opp ansvar for a = 'H', 'U', 'K' eller 'I'
        if a in doc['ansvar']:
            
            # Spesiell behandling dersom rollen har utførende ansvar (a = 'U')
            if a=='U':
                
                table = document.add_table(rows=1, cols=3, style='Table Grid BE-style')
                
                hdr = table.rows[0].cells
                hdr[0].text = stringprep('Skal utføre følgende aktiviteter:')
                hdr[1].text = stringprep('Skal konsultere følgende roller:')
                hdr[2].text = stringprep('Skal informere følgende roller:')
                for d in doc['ansvar']['U'].values():
                    row = table.add_row().cells
                    row[0].text = stringprep(d['desc'])
                    row[1].text = stringprep(', '.join(d['konsultere']))
                    row[2].text = stringprep(', '.join(d['informere']))
                
            else:
                for d in doc['ansvar'][a].values():
                    document.add_paragraph(stringprep(d['desc']), style='Bullet 1')
        else:
            document.add_paragraph(stringprep('Ingen aktiviteter'))
            
    
    document.save(u'Genererte dokumenter/'+stringclean(doc['avdeling'])+'_'+stringclean(doc['rolle'])+u'.docx')
    
    pass     

In [ ]:


for j, rl in enumerate(rolle):
    doc = {'idx': j,
           'rolle': rl,
           'avdeling': avd_rolle[j],
           'hensikt': hensikt[j],
           'overordnet': overord[j],
           'underordnet': underord[j],
           'ansvar': {}}
    for a in ansvar:
        dlist = dict(zip([i for i in xrange(m) if a in data[i,j]], [{'desc': desc[i]} for i in xrange(m) if a in data[i,j]]))
        for i in dlist:
            dlist[i]['konsultere'] = [rolle[k] for k in xrange(n) if 'K' in data[i,k]]
            dlist[i]['informere']  = [rolle[k] for k in xrange(n) if 'I' in data[i,k]]
        if dlist: doc['ansvar'][a] = dlist
    
    generate_document(doc)
    

In [ ]:
doc